##extra functionality for optimization

In [ ]:
#add notr range manualy 
def add_note_range_column(songs_dict, expert_data):
    note_ranges = [songs_dict.get(filename) for filename in expert_data['midi_filename']]
    expert_data['note_range'] = note_ranges
    pass
import json
#for adding note range
# Load dictionary from JSON file
with open('song_dict.json', 'r') as f:
    song_dict = json.load(f)
add_note_range_column(song_dict, expert_data)

In [ ]:
def loss(sum_diff, length):
        #start with avg diff, need to define loss
        return sum_diff / length

In [ ]:
def diff_function(prediction, expert):
    #get two vectors containing orederd predictions and expert data practice modes
    diff = 0
    for i, pred in enumerate(prediction):
        if pred != expert[i]:
            diff+=1
    return loss(diff, len(prediction))
    pass  

In [ ]:
def policy_diff(gauss_model, recorded_points, expert_practice_mode):
         #calculate difference between the prediction of the gauss model and the expert advice.
      r_p =  recorded_points["error_before_right_timing", "error_before_right_pitch"]
      #a question- the diff should be calculated binaricly(by prediction) or by the utility diffs?
      policy_diff =diff_function(gauss_model.get_best_practice_mode(r_p),  expert_practice_mode)
      return policy_diff
      pass


In [ ]:
def policy_diff_utility(gauss_model, point, expert_opt_policy):
    
      c=0#what is c?
      policy_diff= compare_to_best_policy(gauss_model.get_policy(c),point["utility"])
      return policy_diff

In [ ]:
def initialize_gp_generelize():
    #generelized to any number of kernels, not sure if possible
    gp_models = []
    kernel_hyper_params = [{'lengthscale': 1.0}, {'lengthscale': 0.5}, {'lengthscale': 2.0}]
    kernels = [GPy.kern.Matern52, GPy.kern.Matern32, GPy.kern.RBF]

    for kernel, hyper_params in zip(kernels, kernel_hyper_params):
        for param in hyper_params:
            # initialize a new kernel with the given hyperparameters
            kernel_instance = kernel(input_dim=1, **param)

            # initialize a new Gaussian process model with the current kernel
            gp = GaussianProcess(kernel=kernel_instance)

            gp_models.append(gp)

    return gp_models
pass

In [ ]:
def grid_search_opt(expert_data):

    start_time = time.time()
    #gets a utility collumn for expert data
    expert_data["utility"] = expert_data.apply(lambda row: error_diff_to_utility_for_opt(row["error_before_right_pitch"],
                                                                                row["error_after_right_pitch"],
                                                                                row["error_before_right_timing"],
                                                                                row["error_after_right_timing"]),
                                                axis=1)

    #currently just right timing and right pitch
    recorded_points = expert_data[["error_before_right_timing", "error_before_right_pitch","practice_mode", "bpm","utility"]]
    #initilizing the gp models with different hyperparameters
    gauss_models = initilize_gp()
    #calculating the policy diff for each gp, for each added data point
    policy_diff = policy_diff_for_all_gauss_models(gauss_models, recorded_points)
    #choose the optimal gp
    best_gp = optimal_gp(gauss_models, policy_diff)

    end_time = time.time()
    time_elapsed = end_time - start_time
    print("Time elapsed: {:.2f} seconds".format(time_elapsed))
    return best_gp
    pass


In [ ]:
def initilize_gp():
        gp_models = []
        variance = [0.2]
        lengthscale = [1]
        alpha_list = [0.1]
        #RBF kernel
        for v in variance:
            for l in lengthscale:
                #for each kernel, we initialize a gp model with different hyperparameters
                #initial a gp instance
                gp = GaussianProcess()
                kernel = GPy.kern.RBF(input_dim=gp.space.model_dimensionality, 
                                variance=v, 
                                lengthscale=l)
                #we use the update_model_with_kernel function to create the new model with the new kernel
                gp.update_model_with_kernel(kernel)
                gp_models.append(gp)
        #MATERN52 kernel
        for v in variance:
            for l in lengthscale:
                #for each kernel, we initialize a gp model with different hyperparameters
                #initial a gp instance
                gp = GaussianProcess()
                kernel = GPy.kern.Matern52(input_dim=gp.space.model_dimensionality, 
                              variance=v, 
                              lengthscale=l)
                #we use the update_model_with_kernel function to create the new model with the new kernel
                gp.update_model_with_kernel(kernel)
                gp_models.append(gp)
        #Rational Quadratic kernel
        for v in variance:
            for l in lengthscale:
                for alpha in alpha_list:
                    # initialize a gp instance
                    gp = GaussianProcess()
                    kernel = GPy.kern.RatQuad(input_dim=gp.space.model_dimensionality, 
                                            variance=v, 
                                            lengthscale=l, 
                                            power=alpha)
                    #we use the update_model_with_kernel function to create the new model with the new kernel
                    gp.update_model_with_kernel(kernel)
                    gp_models.append(gp)


        #linear kernel
        for v in variance:
            for l in lengthscale:
        # initialize a gp instance
                gp = GaussianProcess()
                kernel = GPy.kern.Linear(input_dim=gp.space.model_dimensionality, 
                                        variances=np.ones(gp.space.model_dimensionality)*v, 
                                        ARD=True, # set to True for Automatic Relevance Determination
                                        )
                kernel.lengthscale = np.ones(gp.space.model_dimensionality)*l
                #we use the update_model_with_kernel function to create the new model with the new kernel
                gp.update_model_with_kernel(kernel)
                gp_models.append(gp)



        return gp_models

In [ ]:
def policy_diff_for_all_gauss_models(gauss_models, recorded_points):
    policy_diff = []
    #for each gp, we calculate the policy diff for each added data point
    for i, gauss_model in enumerate(gauss_models):
        policy_diff.append(policy_diff_for_single_gp(gauss_model, recorded_points))
    #we retrun an array of arrays- array contains the policy diff for a single gp, for each added data point
    return policy_diff